In [ ]:
from pathlib import Path

from one.api import One
from one.params import CACHE_DIR_DEFAULT

# Imports for downloading example data
import requests
from io import BytesIO
import zipfile

# Data locations
cache_dir = Path(CACHE_DIR_DEFAULT, 'ibl_behavior_paper')
data_url = 'https://ndownloader.figshare.com/files/21623715'


The following cell will download some example data from FigShare.  The dataset is around 74,000
behaviour files (~218MB) from the beta data release for the IBL behaviour paper.

In [ ]:
# Download data if necessary
if not (cache_dir.exists() and any(cache_dir.iterdir())):
    cache_dir.parent.mkdir(exist_ok=True, parents=True)  # Create destination dir
    print(f'Downloading data from {data_url.split(".", maxsplit=1)[-1]}...')
    request = requests.get(data_url)  # Download data into memory (~300MB)
    with zipfile.ZipFile(BytesIO(request.content)) as zipped:
        print(f'Extracting into {cache_dir}...')
        zipped.extractall(path=cache_dir.parent)  # Decompress into destination dir
    Path(cache_dir.parent, 'ibl-behavioral-data-Dec2019').rename(cache_dir)  # Rename
    cache_dir.joinpath('one_example.py').unlink(missing_ok=True)  # Delete outdated example
    del request  # Free resources

The following cell will index the files and build the cache (~3 min).  This only needs doing
once.  For a data release these small cache files would typically be provided with the data.

In [ ]:
print('Building ONE cache from filesystem...')
One.setup(cache_dir, hash_files=False)

Now the files can be searched and loaded via the API

In [ ]:
one = One(cache_dir=cache_dir)

## Searching for behaviour experiment by subject
subject = 'NYU-01'
eids = one.search(subject=subject, dataset=['_ibl_trials.intervals.npy',])
print(f'There are {len(eids)} behaviour sessions for subject "{subject}"')

## Regex is supported in the search
subject = 'NYU-.*'
eids = one.search(subject=subject, dataset=['_ibl_trials.intervals.npy',])
print(f'There are {len(eids)} behaviour sessions with {subject} subjects')


In [ ]:
# Loading the data
eid, *eids = eids  # Assign first experiment ID in list to `eid` var

# Load the trials object
trials = one.load_object(eid, 'trials')

# Load an individual dataset
goCue_times = one.load_session_dataset(eid, '_ibl_trials.goCue_times.npy')